In [1]:
import os
import pandas as pd
from fixblank import fixblankname
from fixwords import fixranks, fixphd, fixuni, fixstatus
from fixnames import fix_name
from fixtitle import fix_title
pd.set_option('display.max_rows', 10)
fn = 'บัญชีรายชื่อผู้ทรงคุณวุฒิ_ส.ก.อ..xlsx'
output_file = os.path.splitext(os.path.basename(fn))[0] + '_broken.xlsx'
b = pd.ExcelFile(fn)

In [2]:
cols = ['ดร.', 'ตำแหน่ง',
       'ชื่อ', 'สกุล', 'ตรี', 'ตรี2', 'ตรี3', 'โท1', 'โท2', 'โท3', 'เอก',
       'เอก2', 'ป.ชั้นสูง', 'ป.ชั้นสูง2', 'เชี่ยวชาญ1', 'เชี่ยวชาญ2',
       'เชี่ยวชาญ3', 'เชี่ยวชาญ4', 'เชี่ยวชาญ5',
       'ม/ส', 'สถานภาพ', 'ที่อยู่', 'โทร', 'e-mail']

In [3]:
def edu(a):
    b = ' '.join(a[0:3])
    m = ' '.join(a[3:6])
    phd = ' '.join(a[6:8])
    sp = ' '.join(a[8:])
    return 'ตรี: {}, โท: {}, เอก: {}, ป.ขั้นสูง: {}'.format(b, m, phd, sp)

def skill(a):
    return ', '.join(a[8:])

#d[cols[5:15]].fillna('').apply(edu, axis=1)
#d[cols[16:21]].fillna('').apply(skill, axis=1)

In [4]:
dfs = None
with pd.ExcelWriter(output_file) as writer:
    for c in b.sheet_names:
        print(c)
        d = b.parse(c)
        d = d.dropna(how='all')
        d.columns = [c.strip() for c in d.columns]
        d_broken = fixblankname(d, cols[2:4])
        d_broken.to_excel(writer, sheet_name=c)
        
        d = d[cols]
        d['คุณวุฒิ'] = d[cols[4:14]].fillna('').apply(edu, axis=1)
        d['เชี่ยวชาญ'] = d[cols[14:19]].fillna('').apply(skill, axis=1)
        d = d.drop(cols[4:19], axis=1)
        d['สาขาวิชา'] = c
    
        dfs = pd.concat([dfs, d])
#dfs


สาขาวิชาสังคมศาสตร์
สาขาวิชาศึกษาศาสตร์
สาขาวิชามนุษยศาสตร์
สาขาวิจิตรศิลป์
สาขานิติศาสตร์
สาขาวิทยาศาสตร์
สาขาแพทยศาสตร์
สาขาทันตแพทย์
สาขาพยาบาล
สาขาเภสัชศาสตร์
สาขาสถาปัตยกรรมศาสตร์
สาขาวิศวกรรมศาสตร์
สาขาอื่นๆ
สาขาเกษตศาสตร์


In [5]:
dfs_cols = ['ตำแหน่ง', 'ดร.', 'ชื่อ', 'สกุล', 'คุณวุฒิ', 'ม/ส', 'สาขาวิชา', 'เชี่ยวชาญ', 'ที่อยู่', 'โทร', 'e-mail', 'สถานภาพ']
new_cols = ['ตำแหน่ง', 'ตำแหน่ง ป.เอก', 'ชื่อ', 'นามสกุล', 'คุณวุฒิ', 'มหาวิทยาลัย', 'สาขาวิชา', 'ความเชี่ยวชาญ', 'สถานที่ติดต่อ', 'โทร', 'email', 'สถานะ']
dfs2 = dfs[dfs_cols]
dfs2.columns = new_cols

In [6]:
dfs2['ตำแหน่ง'] = dfs2['ตำแหน่ง'].fillna('').apply(fixranks)
dfs2['ตำแหน่ง ป.เอก'] = dfs2['ตำแหน่ง ป.เอก'].fillna('').apply(fixphd)
dfs2['มหาวิทยาลัย'] = dfs2['มหาวิทยาลัย'].fillna('').apply(fixuni)
dfs2['สถานะ'] = dfs2['สถานะ'].fillna('').apply(fixstatus)
dfs2['ชื่อ'] = dfs2['ชื่อ'].fillna('').apply(fix_name)
dfs2['ชื่อ'] = dfs2['ชื่อ'].fillna('').apply(fix_title)

In [7]:
dfs2

,ตำแหน่ง,ตำแหน่ง ป.เอก,ชื่อ,นามสกุล,คุณวุฒิ,มหาวิทยาลัย,สาขาวิชา,ความเชี่ยวชาญ,สถานที่ติดต่อ,โทร,email,สถานะ
0,รองศาสตราจารย์,ดร.,วาสิตา,บุญสาธร,"ตรี: ภาษาอังกฤษ , โท: Speech Communication ,...",สถาบันพัฒนาบุคลากรท้องถิ่น,สาขาวิชาสังคมศาสตร์,,NaN,0 2727 3324\n0 2727 3490-91,NaN,
1,รองศาสตราจารย์,ดร.,กมลพร,กัลยาณมิตร,"ตรี: รัฐศาสตร์ , โท: รัฐประศาสนศาสตร์ , เอก:...",มหาวิทยาลัยกรุงเทพธนบุรี,สาขาวิชาสังคมศาสตร์,,NaN,0 2431 5383\n0 2800 6800-5,NaN,
2,รองศาสตราจารย์,ดร.,วีระศักดิ์,จินารัตน์,"ตรี: นิติศาสตร์ , โท: พบ.ม. , เอก: Internati...",มหาวิทยาลัยอีสเทิร์นเอเชีย,สาขาวิชาสังคมศาสตร์,,NaN,0 4528 3770-2\nต่อ 1404,NaN,
3,,,ธวัชชัย,ยงกิตติกุล,"ตรี: B.B.A. , โท: Development Economics Econo...",,สาขาวิชาสังคมศาสตร์,,195/5 อาคารเลครัชดา\nออฟฟิส คอมเพล็กซ์\nอาคาร2...,0 2264 0883\n\n,NaN,
4,รองศาสตราจารย์,,สุนา,สิทธิเลิศประสิทธิ,"ตรี: การบัญชี , โท: การบัญชี , เอก: , ป.ขั้...",มหาวิทยาลัยสุโขทัยธรรมาธิราช,สาขาวิชาสังคมศาสตร์,,\n,0 2504 8181-6,NaN,
...,...,...,...,...,...,...,...,...,...,...,...,...
779,ศาสตราจารย์,,นิตย์ศรี,แสงเดือน,"ตรี: วิทยาศาสตร์ ทางทะเล , โท: พันธุศาสตร์ ,...",มหาวิทยาลัยเกษตรศาสตร์,สาขาเกษตศาสตร์,,NaN,"0 7553 8034, 081 380 2524",NaN,เกษียณ
780,ศาสตราจารย์,,ธีระ,เอกสมทราเมษฐ์,"ตรี: เกษตรศาสตร์ , โท: เกษตรศาสตร์ , เอก: Do...",มหาวิทยาลัยสงขลานครินทร์,สาขาเกษตศาสตร์,,NaN,0 7428 6138-9,NaN,
781,ศาสตราจารย์,,ธีรภาพ,เจริญวิริยะภาพ,"ตรี: เกษตรศาสตร์ , โท: ชีววิทยาสภาวะ แวดล้อม ...",มหาวิทยาลัยเกษตรศาสตร์,สาขาเกษตศาสตร์,,NaN,0 2942 7131,NaN,
782,ศาสตราจารย์,,บัญชา,สมบูรณ์สุข,"ตรี: โรคพืช , โท: ส่งเสริมการเกษตร , เอก: Ag...",มหาวิทยาลัยสงขลานครินทร์,สาขาเกษตศาสตร์,,NaN,"0 7428 6120, 0 7428 6134",NaN,


In [8]:
dfs2.to_csv('MUAlist.csv', encoding='utf-8-sig')